In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install streamlit
!pip install pypdf
!pip install pydantic==1.10.9
!pip install langchain
!pip install accelerate
!pip install bitsandbytes
!pip install transformers
!pip install sentence_transformers
!pip install faiss_gpu
!pip install ctransformers
!pip install streamlit_chat
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 32.9 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19580 sha256=0d6004adde3783cd2f2f79e7de18b941262d8199df11156ae3548e7bd1b32b45
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators
  Attempting uninstall: tzlocal
    Found existing installation: tzlocal 5.0.1
    Uninstalling tzlocal

In [ ]:
%%writefile utils.py

from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
FAISS_PATH = 'vector/similarity_db'
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': 'cuda'})
def store_vector(file_path):
    '''
    function will first load the CSV and converts into the embeddings
    by hugging face. FAISS is use to calculate the similarity of embeddings
    which will be store in a vector file
    '''
    loader = CSVLoader(file_path=file_path,
                      encoding="utf-8", csv_args={
                      'delimiter': '\t'})
    data = loader.load()
    db = FAISS.from_documents(data, embeddings)
    db.save_local(FAISS_PATH)

def load_vector():
    db = FAISS.load_local(FAISS_PATH, embeddings)
    return db

def load_llm():
    # Load the locally downloaded model here
    llm = CTransformers(
        model = "drive/MyDrive/llama-2-7b-chat.ggmlv3.q4_0.bin",
        model_type="llama",
        max_new_tokens = 512,
        temperature = 0
    )
    return llm

Writing utils.py


In [ ]:
# store_vector("emp.csv")

In [ ]:
%%writefile app.py


import streamlit as st
from utils import load_llm, load_vector
from langchain.chains import ConversationalRetrievalChain
from streamlit_chat import message

st.title("Employee Book Bot based on CSV")
st.markdown("<h3 style='text-align: center; color: black;'>Built by <a href='https://github.com/hassanrehman11'>Hassan Rehman </a></h3>", unsafe_allow_html=True)

db = load_vector()
llm = load_llm()
chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=db.as_retriever())

def conversational_chat(query):
    result = chain({"question": query, "chat_history": st.session_state['history']})
    st.session_state['history'].append((query, result["answer"]))
    return result["answer"]

if 'history' not in st.session_state:
    st.session_state['history'] = []

if 'generated' not in st.session_state:
    st.session_state['generated'] = ["Hello I am an Employee Bot. You can ask anything related to resources!"]

if 'past' not in st.session_state:
    st.session_state['past'] = ["Hello!!!"]

#container for the chat history
response_container = st.container()
#container for the user's text input
container = st.container()

with container:
    with st.form(key='my_form', clear_on_submit=True):

        user_input = st.text_input("Query:", placeholder="Ask any thing about Pokemon", key='input')
        submit_button = st.form_submit_button(label='Send')

    if submit_button and user_input:
        output = conversational_chat(user_input)

        st.session_state['past'].append(user_input)
        st.session_state['generated'].append(output)

if st.session_state['generated']:
    with response_container:
        for i in range(len(st.session_state['generated'])):
            message(st.session_state["past"][i], is_user=True, key=str(i) + '_user', avatar_style="big-smile")
            message(st.session_state["generated"][i], key=str(i), avatar_style="thumbs")


Writing app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

[..................] | fetchMetadata: sill resolveWithNewModule localtunnel@2.0


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://130.211.197.243:8501

npx: installed 22 in 4.731s
your url is: https://wise-lions-think.loca.lt
2023-08-09 14:00:48.789932: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
